## Instalação Dependencias

In [1]:
pip install PyQt5


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Classes

In [2]:
class Window:
    x_min: int
    y_min: int
    x_max: int
    y_max: int
    
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

class Viewport:
    x_min: int
    y_min: int
    x_max: int
    y_max: int
    
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        
class Ponto:
    x: int
    y: int
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

class Reta:
    def __init__(self, ponto1, ponto2):
        self.ponto1 = ponto1
        self.ponto2 = ponto2
        
class Poligono:
    def __init__(self, pontos):
        self.pontos = pontos 

## Transformação

In [3]:
def transformacao(viewport: Viewport, window: Window, ponto: Ponto):
    x_vp = ((ponto.x - window.x_min)/(window.x_max - window.x_min))*(viewport.x_max - viewport.x_min)
    y_vp = (1 - ((ponto.y - window.y_min)/(window.y_max - window.y_min)))*(viewport.y_max - viewport.y_min)
        
    return Ponto(x_vp, y_vp)

## Parser XML

In [4]:
import xml.etree.ElementTree as ET

def parse_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()

    objetos = []
    
    vp_min = root.find("./viewport/vpmin")
    vp_max = root.find("./viewport/vpmax")
    viewport = Viewport(int(vp_min.attrib['x']), int(vp_min.attrib['y']), int(vp_max.attrib['x']), int(vp_max.attrib['y']))

 
    w_min = root.find("./window/wmin")
    w_max = root.find("./window/wmax")
    window = Window(float(w_min.attrib['x']), float(w_min.attrib['y']), float(w_max.attrib['x']), float(w_max.attrib['y']))

    for ponto in root.findall("./ponto"):
        objetos.append(Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))

    for reta in root.findall("./reta"):
        ponto1 = reta.find("ponto[1]")
        ponto2 = reta.find("ponto[2]")
        objetos.append(Reta(Ponto(float(ponto1.attrib['x']), float(ponto1.attrib['y'])), Ponto(float(ponto2.attrib['x']), float(ponto2.attrib['y']))))

    for poligono in root.findall("./poligono"):
        pontos = []
        for ponto in poligono.findall("ponto"):
            pontos.append(Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))
        objetos.append(Poligono(pontos))
        
    return viewport, window, objetos

## Viewport transform objects

In [5]:
class viewportTransform():
    def apply_transform(self, window, viewport, objects):
        transformed_objects = []
        
        for objeto in objects:
            if isinstance(objeto, Ponto):
                transformed_objects.append(transformacao(viewport, window, objeto))
            elif isinstance(objeto, Reta):
                objeto.ponto1 = transformacao(viewport, window, objeto.ponto1)
                objeto.ponto2 = transformacao(viewport, window, objeto.ponto2)
                transformed_objects.append(objeto)
            elif isinstance(objeto, Poligono):
                pontos_transformados = [transformacao(viewport, window, ponto) for ponto in objeto.pontos]
                objeto.pontos = pontos_transformados
                transformed_objects.append(objeto)

        return transformed_objects
    

## Viewport

In [6]:
%matplotlib qt
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow
from PyQt5.QtGui import QPainter, QPen, QPixmap
from PyQt5.QtCore import Qt

In [7]:
class Render(QMainWindow):
    def __init__(self, viewport: Viewport):
        super().__init__()
        
        self.setWindowTitle("Computação Gráfica")
        self.setGeometry(viewport.x_min-20, viewport.y_min-20, viewport.x_max+20, viewport.y_max+20)

        #viewport
        self.label = QLabel(self)
        self.label.setGeometry(viewport.x_min-10, viewport.y_min-10, viewport.x_max+10, viewport.y_max+10)
        
        #desenho
        self.pixmap = QPixmap(self.label.size())
        self.pixmap.fill(Qt.white)
        self.painter = QPainter(self.pixmap)
        pen = QPen(Qt.black)
        pen.setWidth(2)
        self.painter.setPen(pen)
        
    def draw_line(self, reta:Reta):
        pixmap = self.label.pixmap()
        if not pixmap:
            pixmap = QPixmap(self.label.size())
            pixmap.fill(Qt.white)

        painter = QPainter(pixmap)
        pen = QPen(Qt.black)
        pen.setWidth(2)
        painter.setPen(pen)
        
        painter.drawLine(int(reta.ponto1.x), int(reta.ponto1.y), int(reta.ponto2.x), int(reta.ponto2.y))
        painter.end()

        self.label.setPixmap(pixmap)
        
    def draw_polygon(self, poligono:Poligono):
            pontoOrigin = poligono.pontos[0]
            
            for i in range(0, int(len(poligono.pontos))):
                if i == len(poligono.pontos)-1:
                    self.draw_line(Reta(poligono.pontos[i], pontoOrigin))
                else:
                    self.draw_line(Reta(poligono.pontos[i], poligono.pontos[i+1]))
                
    def draw_point(self, ponto:Ponto):
        pixmap = self.label.pixmap()
        if not pixmap:
            pixmap = QPixmap(self.label.size())
            pixmap.fill(Qt.white)

        painter = QPainter(pixmap)
        pen = QPen(Qt.black)
        pen.setWidth(2)
        painter.setPen(pen)
        
        painter.drawPoint(int(ponto.x), int(ponto.y))
        painter.end()
        
        self.label.setPixmap(pixmap)
    
    def render_objects(self, objects):
        for object in objects:
            if isinstance(object, Ponto):
                self.draw_point(object)
            elif isinstance(object, Reta):
                self.draw_line(object)
            elif isinstance(object, Poligono):
                self.draw_polygon(object)

## Main

In [8]:

class main():
    viewport, window, objetos = parse_xml("./input/entrada.xml")

    transform = viewportTransform()
    transformed_object = transform.apply_transform(window, viewport, objects=objetos)
    
    window = Render(viewport)
    window.render_objects(transformed_object)

    window.show()
    
if __name__ == 'main':
    main()